In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Update the path to include the data directory
amazon_sales = '../data/Amazon Sale Report.csv'

# Now read the CSV file from the data directory
sales = pd.read_csv(amazon_sales)


#CLEANING: 

sales = sales.drop(columns=['fulfilled-by', 'Unnamed: 22'])


# Function to display initial data overview
def data_overview(df, name):
    print(f"\nOverview of {name} dataset:")
    print(df.info())
    print(df.head())
    print(f"\nNumber of duplicates in {name}: {df.duplicated().sum()}")
    print(f"Missing values in {name}:\n{df.isnull().sum()}\n")
    
# Replace missing values in 'Courier Status' with 'Unknown' since 5.33% are null
sales['Courier Status'].fillna('Unknown', inplace=True)

# Replace missing values in 'currency' with 'INR' since 6.04% are null & all non-null are INR
sales['currency'].fillna('INR', inplace=True)

# Calculate the mean and median of the 'Amount' column 
amount_mean = sales['Amount'].mean()
amount_median = sales['Amount'].median()
# Replace missing values in 'Amount' with the mean since 6.04% are null
sales['Amount'].fillna(amount_mean, inplace=True)

# Replace missing values with 'Unknown'
columns_to_fill = ['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']

for column in columns_to_fill:
    sales[column].fillna('Unknown', inplace=True)

# Replace NaN values in 'promotion-ids' with 'None'
sales['promotion-ids'] = sales['promotion-ids'].fillna('None')


sales['Date'] = pd.to_datetime(sales['Date'], errors='coerce')
sales['Fulfilment'] = sales['Fulfilment'].astype('category')
sales.columns = sales.columns.str.strip()

# List of columns to convert to 'category' type
columns_to_convert = ['Fulfilment', 'Sales Channel', 'ship-service-level']

# Convert the columns to 'category' type
sales[columns_to_convert] = sales[columns_to_convert].astype('category')
     
    
# Display initial data overviews
data_overview(sales, "sales")



/Users/juanescobar/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



Overview of sales dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  datetime64[ns]
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  category      
 5   Sales Channel       128975 non-null  category      
 6   ship-service-level  128975 non-null  category      
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier Status      128975 non-null  object        
 13  Q

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore


# Make sure the 'visualizations' directory exists
import os
if not os.path.exists('../visualizations'):
    os.makedirs('../visualizations')

# Plot 1: Distribution Of 'Amount' In Sales
plt.figure(figsize=(10, 6))
plt.hist(sales['Amount'], bins=50, color='blue', alpha=0.5, label='All Data')
plt.title("Histogram of 'Amount'")
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.legend()
plt.savefig('../visualizations/Histogram_Amount.png')
plt.close()

# Find outliers of 'Amount'
sales['z_score'] = zscore(sales['Amount'])
outliers = sales[sales['z_score'].abs() > 3]

# Plot 2: Comparison of Distribution: All Data vs Outliers
plt.figure(figsize=(10, 6))
plt.hist(sales['Amount'], bins=50, color='blue', alpha=0.5, label='All Data')
plt.hist(outliers['Amount'], bins=50, color='red', alpha=0.7, label='Outliers')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.title('Comparison of Distribution: All Data vs Outliers')
plt.legend()
plt.savefig('../visualizations/All_Data_vs_Outliers.png')
plt.close()

# Plot 3: Distribution of Amount with Outliers
plt.figure(figsize=(12, 6))
sns.histplot(sales['Amount'], kde=True, bins=50)
plt.title('Distribution of Amount with Outliers')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.savefig('../visualizations/Distribution_Amount_With_Outliers.png')
plt.close()

# Filter out outliers
filtered_sales = sales[sales['Amount'] <= 2000]

# Plot 4: Distribution of Amount without Outliers
plt.figure(figsize=(12, 6))
sns.histplot(filtered_sales['Amount'], kde=True, bins=50)
plt.title('Distribution of Amount without Outliers')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.savefig('../visualizations/Distribution_Amount_Without_Outliers.png')
plt.close()


In [17]:
#Total Sales Per Month


# Plot 5: Filtered Total Sales Per Day
daily_sales = filtered_sales.groupby('Date')['Amount'].sum().reset_index()
min_y_all = daily_sales['Amount'].min()
max_y_all = daily_sales['Amount'].max()

plt.figure(figsize=(12, 6))
plt.plot(daily_sales['Date'], daily_sales['Amount'], marker='o')
plt.title('Filtered Total Sales Per Day')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.ylim(0, max_y_all + 20000)  # Set y-axis limits
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/Total_Sales_Per_Day.png')
plt.close()

# Plot 6: Filtered Total Sales Per Month (Excluding March) / group by month and sum of sales
sales_filtered = sales[~sales['Date'].dt.month.isin([3])]
monthly_sales_filtered = sales_filtered.groupby(sales_filtered['Date'].dt.to_period('M'))['Amount'].sum()

plt.figure(figsize=(12, 6))
monthly_sales_filtered.plot(kind='bar')
plt.title('Filtered Total Sales Per Month (Excluding March)')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.tight_layout()
plt.savefig('../visualizations/Total_Sales_Per_Month_Excluding_March.png')
plt.close()


In [18]:
#SKU Exploration



# Plot 7: Distribution of Top 20 SKUs
sku_counts = sales['SKU'].value_counts()
top_20_skus = sku_counts.head(20).reset_index()
top_20_skus.columns = ['SKU', 'Count']

plt.figure(figsize=(12, 8))
sns.barplot(x='SKU', y='Count', data=top_20_skus, palette='viridis')
plt.title('Distribution of Top 20 SKUs')
plt.xlabel('SKU')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('../visualizations/Top_20_SKUs.png')
plt.close()


# Plot 8: Distribution of Top 20 Simplified SKUs
size_suffixes = ['-XS','-S', '-M', '-L', '-XL', '-XXL','-XXXL']


def remove_size_suffix(sku):
    for suffix in size_suffixes:
        if sku.endswith(suffix):
            return sku[:-len(suffix)]
    return sku



sales['Simplified_SKU'] = sales['SKU'].apply(remove_size_suffix)
simplified_sku_counts = sales['Simplified_SKU'].value_counts()
top_simplified_skus = simplified_sku_counts.head(20).reset_index()
top_simplified_skus.columns = ['SKU', 'Count']

plt.figure(figsize=(14, 10))
sns.barplot(x='SKU', y='Count', data=top_simplified_skus, palette='viridis')
plt.title('Distribution of Top 20 Simplified SKUs')
plt.xlabel('SKU')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('../visualizations/Top_20_Simplified_SKUs.png')
plt.close()


#___________________________________________________________

#

In [20]:
#SEGMENTATION BEGIN

# Define broader categories
category_mapping = {
    'Set': 'Accessories',
    'kurta': 'Ethnic Wear',
    'Western Dress': 'Western Wear',
    'Top': 'Western Wear',
    'Ethnic Dress': 'Ethnic Wear',
    'Bottom': 'Accessories',
    'Saree': 'Ethnic Wear',
    'Blouse': 'Accessories',
    'Dupatta': 'Ethnic Wear'
}

# Plot 9: Total Sales by Segment
sales['Segment'] = sales['Category'].map(category_mapping)
# Calculate total sales by segment
segment_sales = sales.groupby('Segment')['Amount'].sum()

plt.figure(figsize=(10, 6))
sns.barplot(x=segment_sales.index, y=segment_sales.values)
plt.title('Total Sales by Segment')
plt.ylabel('Total Sales')
plt.xlabel('Segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../visualizations/Total_Sales_by_Segment.png')
plt.close()

# Plot 10: Top 20 Simplified SKUs by Total Amount (Categorized)
sku_amounts = sales.groupby(['Simplified_SKU', 'Category'])['Amount'].sum().reset_index(name='Total_Amount')
top_20_skus = sku_amounts.nlargest(20, 'Total_Amount')
palette = sns.color_palette("husl", len(top_20_skus['Category'].unique()))

plt.figure(figsize=(14, 10))
sns.barplot(x='Total_Amount', y='Simplified_SKU', hue='Category', data=top_20_skus, palette=palette)
plt.title('Top 20 Simplified SKUs by Total Amount (Categorized)')
plt.xlabel('Total Amount')
plt.ylabel('Simplified SKU')
plt.legend(title='Category', loc='right')
plt.tight_layout()
plt.savefig('../visualizations/Top_20_Simplified_SKUs_By_Total_Amount.png')
plt.close()





